In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_ini=pd.read_csv('titanic.csv')

In [3]:
for feature in df_ini.columns[:]:
    if df_ini[feature].dtype!='object':        
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].median(),df_ini[feature])
    else:
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].value_counts().index[0],df_ini[feature])
df_aim=df_ini

In [4]:
def reol(feature1):
    IQR=df_aim[feature1].quantile(0.75)-df_aim[feature1].quantile(0.25)
    lbound=df_aim[feature1].quantile(0.25)-(IQR*1.5)
    ubound=df_aim[feature1].quantile(0.75)+(IQR*1.5)
    df_aim.loc[df_aim[feature1]<=lbound,feature1]=lbound
    df_aim.loc[df_aim[feature1]>=ubound,feature1]=ubound
df_aol=df_aim

In [5]:
for feature in df_aim.columns[:]:
    if df_aim[feature].dtype!='object':
        reol(feature)

In [6]:
df_aol_cont1=df_aol.select_dtypes(exclude=[object])

In [7]:
df_aol_cont2=df_aol_cont1.drop(columns='Pclass',axis=1) #Dropping categorical ordinal features

In [8]:
df_aol_cont3=df_aol_cont2.drop('Survived',axis=1)#varience threshold can be applied for independent features only

In [9]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_aol_cont3)

VarianceThreshold(threshold=0)

In [10]:
const_features=[column for column in df_aol_cont3.columns\
               if column not in df_aol_cont3.columns[var_thres.get_support()]]

In [11]:
df_aol=df_aol.drop(const_features,axis=1)#Dropping constant features

In [12]:
df_aol_cont3=df_aol_cont3.drop(columns=const_features,axis=1)

In [13]:
X=df_aol_cont3
y=df_aol_cont2['Survived']

In [14]:
def pcc(dataset,threshold):
    dropset=set()
    cor_mat=dataset.corr()
    for i in range(len(cor_mat.columns)):
        for j in range(i):
            if abs(cor_mat.iloc[i,j])>=threshold:
                cor_feat=cor_mat.columns[i]
                dropset.add(cor_feat)
    return dropset 
dropset1=pcc(X,0.85)
print("The Features to be Dropped are...")
dropset1

The Features to be Dropped are...


set()

In [15]:
df_aol=df_aol.drop(columns=dropset1,axis=1)

In [16]:
def enc(feature1):
    key_labels=df_aol[feature1].unique()
    dict={k:i for i,k in enumerate(key_labels,0)}
    df_aol[feature1]=df_aol[feature1].map(dict)

In [17]:
for feature in df_aol.columns[:]:
    if df_aol[feature].dtype=='object':
        enc(feature)
df_aec=df_aol

In [18]:
from sklearn.feature_selection import mutual_info_classif
X=df_aec.drop(columns='Survived',axis=1)
y=df_aec['Survived']
mut_inf_data=mutual_info_classif(X,y,random_state=0)
mut_inf_modif=pd.Series(mut_inf_data,index=X.columns)
mut_inf_sorted=mut_inf_modif.sort_values(ascending=False)
dict=mut_inf_sorted.to_dict()

In [19]:
droplist=[]
for key in dict:
    if dict[key]<0.001:
        droplist.append(key)

In [20]:
df_aec=df_aec.drop(columns=droplist,axis=1)

In [21]:
df_aim_cate1=df_aim.select_dtypes(include=[object])#For imbalanced data check,include all categorical features+Dep. feature

In [22]:
df_aim_cate2=df_aim[['Pclass','Survived']].copy()#include categorical ordinal feature and
df_aim_cate=pd.concat([df_aim_cate1,df_aim_cate2],axis=1)#dependent feature in the new Data frame.

In [23]:
imb_chk_list=list()

In [24]:
for feature in df_aim_cate.columns[:]:
    length=int(len(df_aim_cate[feature].unique()))      
    if length <= 5:
        imb_chk_list.append(feature)

In [25]:
drop_list=[]#Add the independent features with single category, imbalanced data in this list(Difference of values<0.05 and >0.95)

In [26]:
df_aec=df_aec.drop(drop_list,axis=1)

In [27]:
original_list=['Survived','Pclass','Sex','SibSp','Cabin','Embarked']
log_list=['Fare']
expo_list=[]
sqroot_list=['PassengerId','Name','Ticket']
boxcox_list=[]

In [28]:
for feature in log_list:
    df_aec[feature]=np.log1p(df_aec[feature])

In [29]:
for feature in sqroot_list:
    df_aec[feature]=df_aec[feature]**(1/2)

In [30]:
df_atr=df_aec

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled=scaler.fit_transform(df_atr)

In [32]:
df_ast=pd.DataFrame(scaled,columns=df_atr.columns)

In [33]:
df_fin=df_ast

In [34]:
X=df_fin.drop(columns='Survived',axis=1)
y=df_fin['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=0)

In [37]:
from sklearn import svm
classifier=svm.SVC()
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[151  17]
 [ 35  65]]
Accuracy Score : 0.8059701492537313
              precision    recall  f1-score   support

         0.0       0.81      0.90      0.85       168
         1.0       0.79      0.65      0.71       100

    accuracy                           0.81       268
   macro avg       0.80      0.77      0.78       268
weighted avg       0.80      0.81      0.80       268



In [39]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal,uniform
classifier=svm.SVC()

In [40]:
param_grid={'C':[0.1,1,10,100],
            'gamma':[1,0.1,0.01,0.001],
            'kernel':['rbf','poly','sigmoid']}            

In [41]:
grid_search=GridSearchCV(estimator=classifier,param_grid=param_grid,cv=10,
                         verbose=2,n_jobs=-1)

In [42]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   25.6s finished


GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=2)

In [43]:
best_grid=grid_search.best_estimator_
best_grid

SVC(C=10, gamma=0.1)

In [44]:
prediction=best_grid.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[144  24]
 [ 32  68]]
Accuracy Score : 0.7910447761194029
              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       168
         1.0       0.74      0.68      0.71       100

    accuracy                           0.79       268
   macro avg       0.78      0.77      0.77       268
weighted avg       0.79      0.79      0.79       268



In [46]:
# pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    C=trial.suggest_categorical('C',[0.1,1,10,100])
    gamma=trial.suggest_categorical('gamma',[1,0.1,0.01,0.001])
    kernel=trial.suggest_categorical('kernel',['rbf','poly','sigmoid'])
    classifier=svm.SVC(C=C,gamma=gamma,kernel=kernel)
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=3).mean()
    return score

In [47]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2021-07-30 23:48:53,863] A new study created in memory with name: no-name-f91f347d-23b6-4288-b135-1e86556dac96
[I 2021-07-30 23:48:53,903] Trial 0 finished with value: 0.6517016598538338 and parameters: {'C': 100, 'gamma': 1, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6517016598538338.
[I 2021-07-30 23:48:53,940] Trial 1 finished with value: 0.6901244890375325 and parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}. Best is trial 1 with value: 0.6901244890375325.
[I 2021-07-30 23:48:54,159] Trial 2 finished with value: 0.772064288368636 and parameters: {'C': 10, 'gamma': 1, 'kernel': 'poly'}. Best is trial 2 with value: 0.772064288368636.
[I 2021-07-30 23:48:54,195] Trial 3 finished with value: 0.7865725257029604 and parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}. Best is trial 3 with value: 0.7865725257029604.
[I 2021-07-30 23:48:54,225] Trial 4 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 w

[I 2021-07-30 23:48:55,796] Trial 41 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:55,838] Trial 42 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:55,872] Trial 43 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:55,907] Trial 44 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:55,939] Trial 45 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:55,975] Trial 46 finished with value: 0.7897931376192245 and

[I 2021-07-30 23:48:57,730] Trial 83 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:57,762] Trial 84 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:57,794] Trial 85 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:57,826] Trial 86 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:57,861] Trial 87 finished with value: 0.7978214418431809 and parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}. Best is trial 4 with value: 0.7978214418431809.
[I 2021-07-30 23:48:57,901] Trial 88 finished with value: 0.7978214418431809 and

In [48]:
best=study.best_params

In [49]:
best

{'C': 10, 'gamma': 0.1, 'kernel': 'poly'}

In [50]:
classifier=svm.SVC(C=best['C'],gamma=best['gamma'],kernel=best['kernel'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [51]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[150  18]
 [ 35  65]]
Accuracy Score : 0.8022388059701493
              precision    recall  f1-score   support

         0.0       0.81      0.89      0.85       168
         1.0       0.78      0.65      0.71       100

    accuracy                           0.80       268
   macro avg       0.80      0.77      0.78       268
weighted avg       0.80      0.80      0.80       268



In [52]:
# pip install hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
space={'C':hp.choice('C',[0.1,1,10,100]),
       'gamma':hp.choice('gamma',[1,0.1,0.01,0.001]),
       'kernel':hp.choice('kernel',['rbf','poly','sigmoid'])}

In [53]:
def objective(space):
    classifier=svm.SVC(C=space['C'],gamma=space['gamma'],kernel=space['kernel'])
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=5).mean()
    return{'loss':-score,'status':STATUS_OK}

In [54]:
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|████████████████████████████████████████████████| 80/80 [00:08<00:00,  9.99trial/s, best loss: -0.809058064516129]


{'C': 2, 'gamma': 1, 'kernel': 0}

In [55]:
ce={0:0.1,1:1,2:10,3:100}
gam={0:1,1:0.1,2:0.01,3:0.001}
ker={0:'rbf',1:'poly',2:'sigmoid'}

In [56]:
classifier=svm.SVC(C=ce[best['C']],gamma=gam[best['gamma']],kernel=ker[best['kernel']])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[144  24]
 [ 32  68]]
Accuracy Score : 0.7910447761194029
              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       168
         1.0       0.74      0.68      0.71       100

    accuracy                           0.79       268
   macro avg       0.78      0.77      0.77       268
weighted avg       0.79      0.79      0.79       268

